<b>
    
<p>
<center>
<font size="5">
Create API for Movie Recommend
</font>
</center>
<center>
<font size="4">
Tạo API cho trang web đề xuất phim
</font>
</center>
</p>
    
</b>

<p><h2>Table of Contents<span class="tocSkip"></span></h2></p>

<div class="toc"><ul class="toc-item">
    <li><span><a ><span>1&nbsp;&nbsp;</span>Overview</a></span></li>
    <li><span><a ><span>2&nbsp;&nbsp;</span>Import Library</a></span></li>
    <li><span><a ><span>3&nbsp;&nbsp;</span>Load Data</a></span></li>
    <li><span><a ><span>4&nbsp;&nbsp;</span>Tạo API cho trang web</a></span></li>

### Overview 

#### Aim 
- Tạo API cho trang web

### Import Library

In [1]:
import os
os.environ["SPARK_HOME"] = "/opt/spark"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"

In [2]:
import findspark
findspark.init()
from pyspark.sql.functions import monotonically_increasing_id 
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

In [3]:
import findspark
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext, SparkConf

spark = SparkSession.builder \
    .appName("Project") \
    .master("local[*]") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "8") \
    .getOrCreate()
sc = spark.sparkContext

23/05/27 14:40:52 WARN Utils: Your hostname, longcule-Inspiron-7591 resolves to a loopback address: 127.0.1.1; using 192.168.1.40 instead (on interface enx68e43b306c5a)
23/05/27 14:40:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/27 14:40:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/27 14:40:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/27 14:40:53 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Load Data

In [4]:

mov = spark.read.csv('hdfs://127.0.0.1:9900/user/movies.csv', header=True,inferSchema="true")
mov.show()


+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [5]:

tags = spark.read.csv('hdfs://127.0.0.1:9900/user/tags_new.csv',header=True).drop('userId')


In [6]:

read_movies_1m = sc.textFile("hdfs://127.0.0.1:9900/user/movies.csv")
read_movies_1m_header = read_movies_1m.take(1)[0]
read_movies_1m_header

'movieId,title,genres'

In [7]:

s_df = spark.read.format("csv").option("header", "true").load('hdfs://127.0.0.1:9900/user/movies.csv')
a = [i.movieId for i in s_df.select('movieId').distinct().collect()]


In [8]:

read_tags_1m = sc.textFile("hdfs://127.0.0.1:9900/user/tags_new.csv")
read_tags_1m_header = read_tags_1m.take(1)[0]
read_tags_1m_header

23/05/27 14:41:03 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3025)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

'movieId,tag'

In [9]:
#creation of document for each movie in movies tags
read_tags_1m_data = read_tags_1m.filter(lambda line : line != read_tags_1m_header).map(lambda line:line.split(",")).map(lambda tokens:(int(tokens[0]),tokens[1]))\
.groupByKey().sortByKey()

documents = read_tags_1m_data.map(lambda x :   [i  for i in x[1]])

read_tags = spark.createDataFrame(read_tags_1m_data,('movieid_num', 'document'))

a = [i.movieid_num for i in read_tags.select('movieid_num').distinct().collect()]


In [10]:

tag_movies = read_tags.join(mov, read_tags.movieid_num == mov.movieId).drop('movieId')
tag_movies_id = tag_movies.select("*").withColumn("id", monotonically_increasing_id())
tag_movies.count()


57716

In [11]:
a.sort()
a.index(109487)
d1 = documents.collect()
d1[826]
a.index(236)

233

In [12]:

bagfwords = read_tags_1m.filter(lambda line : line != read_tags_1m_header).map(lambda line:line.split(",")).map(lambda tokens:((tokens[1]), int(1)))\
.reduceByKey(lambda x,y:x+y)
dfbg = spark.createDataFrame(bagfwords,('words','frequency'))
display(dfbg)
features_length = bagfwords.count()
print(bagfwords.count())

DataFrame[words: string, frequency: bigint]

105871


In [13]:
#tags count
read_tags_1m_data.count()

57716

In [14]:
from pyspark.mllib.feature import HashingTF, IDF
hashingTF = HashingTF(features_length)
tf = hashingTF.transform(documents)
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)



In [15]:
def get_movie_docbyid(movieid):
  return read_tags_1m_data.filter(lambda x: x[0] == movieid).map(lambda x :  [i  for i in x[1]])

Make Recommend From movies

In [16]:
import pandas as pd
import random

def random_movie_genre(genres, df):
    matched_movies = df[df['genres'].str.contains(genres, case=False)]

    if matched_movies.empty:
        return None

    movie_ids = matched_movies['movieId'].tolist()
    random_movie_id = random.choice(movie_ids)

    return random_movie_id

# Đọc file CSV
df = pd.read_csv('data.csv')

In [17]:
from pyspark.mllib.feature import Normalizer
from pyspark.mllib.linalg import SparseVector, DenseVector
def get_movie_ids_gen(movie_id):
    
    new_movie = get_movie_docbyid(movie_id)
    candidateTf = hashingTF.transform(new_movie)
    candidateTfIdf = idf.transform(candidateTf)
    
    frequencyDenseVectors_0 = tfidf.map(lambda vector: DenseVector(vector.toArray()))
    frequencyDenseVectors_1 =  candidateTfIdf.map(lambda vector: DenseVector(vector.toArray()))

    y1 =   frequencyDenseVectors_1.collect()
    re = frequencyDenseVectors_0.map(lambda x : (x.dot(y1[0]))/(x.norm(2)*y1[0].norm(2)))

    result1=re.collect()


    dict1 = {}
    list_index  = []
    for i in result1:
      if (i > 0.1)&(i<0.999999) :
        dict1[a[result1.index(i)]] = i
    sorted_d = sorted(dict1.items(), key=lambda x: x[1])
    top5_r  = sorted_d[-20:]
    for i in top5_r:
      list_index.append(i[0])

    return list_index

In [18]:
# Đọc file CSV
movie_gen = pd.read_csv('data.csv')
def random_movie_by_genre(genres):
    
    movieid = random_movie_genre(genres, movie_gen)
    result = get_movie_ids_gen(movieid)
    return result


In [19]:
movies = spark.read.csv("hdfs://127.0.0.1:9900/user/links.csv", header=True, inferSchema=False)

In [20]:

def show_imdbId_gen(genres):
    rec_df = spark.createDataFrame([(i,) for i in random_movie_by_genre(genres)], ['movieId'])
    result = rec_df.join(movies,['movieId'])
    imdbid = result.select(result['imdbId'].cast('string')).rdd.flatMap(lambda x: x).collect()
    imdbid_with_tt = ['tt' + imdbid for imdbid in imdbid]
    return imdbid_with_tt
# genres_1 = 'Drama'
# show_imdbId_gen(genres_1)

Recom movieId

In [21]:
from pyspark.mllib.feature import Normalizer
from pyspark.mllib.linalg import SparseVector, DenseVector
def get_movie_ids(movie_id):
    
    new_movie = get_movie_docbyid(movie_id)
    candidateTf = hashingTF.transform(new_movie)
    candidateTfIdf = idf.transform(candidateTf)
    
    frequencyDenseVectors_0 = tfidf.map(lambda vector: DenseVector(vector.toArray()))
    frequencyDenseVectors_1 =  candidateTfIdf.map(lambda vector: DenseVector(vector.toArray()))

    y1 =   frequencyDenseVectors_1.collect()
    re = frequencyDenseVectors_0.map(lambda x : (x.dot(y1[0]))/(x.norm(2)*y1[0].norm(2)))

    result1=re.collect()


    dict1 = {}
    list_index  = []
    for i in result1:
      if (i > 0.1)&(i<0.999999) :
        dict1[a[result1.index(i)]] = i
    sorted_d = sorted(dict1.items(), key=lambda x: x[1])
    top5_r  = sorted_d[-20:]
    for i in top5_r:
      list_index.append(i[0])

    return list_index

In [22]:

def show_imdbId_movieid(movie_id):
    rec_df = spark.createDataFrame([(i,) for i in get_movie_ids(movie_id)], ['movieId'])
    result = rec_df.join(movies,['movieId'])
    imdbid = result.select(result['imdbId'].cast('string')).rdd.flatMap(lambda x: x).collect()
    imdbid_with_tt = ['tt' + imdbid for imdbid in imdbid]
    return imdbid_with_tt
# movie_id = 1
# show_imdbId_movieid(movie_id)

Recom UserId

In [23]:
recommendation = spark.read.parquet('/home/longcule/Documents/bigdata/recommendation.parquet')

In [24]:
recommendation.show(5)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{183947, 4.70272...|
|     3|[{184299, 5.03557...|
|     5|[{140224, 4.85786...|
|     6|[{156414, 4.93688...|
|     9|[{2858, 4.962009}...|
+------+--------------------+
only showing top 5 rows



In [25]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import mean, col
def get_recommendations(user_id):
    recs = recommendation.filter(col("userId") == user_id).select("recommendations")
    recs = recs.select(explode(col("recommendations")).alias("rec")).select("rec.movieId", "rec.rating")
    item_list = recs.orderBy(col("rating").desc()).select("movieId").rdd.flatMap(lambda x: x).collect()
    return item_list

In [26]:
links = spark.read.csv("hdfs://127.0.0.1:9900/user/links.csv", header=True, inferSchema=False)

def show_imdbId(user_id):
    rec_df = spark.createDataFrame([(i,) for i in get_recommendations(user_id)], ['movieId'])
    result = rec_df.join(links,['movieId'])
    imdbid = result.select(result['imdbId'].cast('string')).rdd.flatMap(lambda x: x).collect()
    imdbid_with_tt = ['tt' + imdbid for imdbid in imdbid]
    return imdbid_with_tt

# show_imdbId(10)


### FLASK create API 

In [27]:
from flask import Flask, jsonify

app = Flask(__name__)


In [28]:
@app.after_request
def add_cors_headers(response):
    response.headers['ngrok-skip-browser-warning'] = '12382' # Cho phép từ tất cả các nguồn truy cập
    return response

In [29]:
@app.route('/recommendations/<int:user_id>', methods=['GET'])
def get_recommendations_api(user_id):
    item_list = show_imdbId(user_id)
    return jsonify({'recommendations': item_list})

In [30]:
@app.route('/recommendations/movieid/<int:movie_id>', methods=['GET'])
def get_imdbid(movie_id):
    imdb_ids = show_imdbId_movieid(movie_id)
    return jsonify({'movieId': imdb_ids})


In [31]:
@app.route('/recommendations/genres/<string:genres>', methods=['GET'])
def get_imdbid_by_genre(genres):
    imdb_ids = show_imdbId_gen(genres)
    return jsonify({'genres': imdb_ids})

In [32]:
if __name__=="__main__":
    app.run(host=os.getenv('IP', '127.0.0.1'), 
            port=int(os.getenv('PORT', 5000
            )))

127.0.0.1 - - [27/May/2023 14:42:20] "GET /recommendations/genres/Drama HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 14:43:04] "GET /recommendations/movieid/1 HTTP/1.1" 200 -
